In [1]:
from matplotlib import pyplot
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from facenet_pytorch import MTCNN
import torch
import cv2
from numpy import asarray, savez_compressed, load, expand_dims, save,array
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import albumentations as A
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

D:\Programming\ML\FYP_PROJECT\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Programming\ML\FYP_PROJECT\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Programming\ML\FYP_PROJECT\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Programming\ML\FYP_PROJECT\myenv\lib\site-packages\tensorflow\python\f

In [2]:
def extract_face(filename, required_size=(224,224)):
    # load image from file
    image = cv2.imread(filename)
    # convert to RGB, if needed
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # convert to array
    pixels = asarray(image)
    # create the detector using default weights
    detector = mtcnn.MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    face_array = []
    # extract the bounding box from the first face
    for res in results:
        x1, y1, width, height = res['box']
        # confidence
        confidence = res['confidence']
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1+width, y1+height
        # greater than 0.95
        if confidence > 0.85:
            # extract the face
            face = pixels[y1:y2, x1:x2]
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array.append(asarray(image))
    return face_array

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [32]:
mtcn = MTCNN(margin=20, post_process=False, device=device)

In [29]:
def extract_fast_face(filename, required_size=(224,224)):
    img = cv2.imread(filename)
    img_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pixels = asarray(img_rgb)
    boxes, _ = mtcn.detect(pixels)
    face_array = []
    for c, i in enumerate(_):
        if i > 0.95:
            box = boxes[c]
            f = img_rgb.crop((box[0], box[1], box[2], box[3]))
            image = f.resize(required_size)
            face_array.append(asarray(image))
    return face_array

In [24]:
def load_faces(directory):
    faces = list()
    #enumerate files
    for filename in os.listdir(directory):
        # path
        path = os.path.join(directory, filename)
        # get face
        face = extract_fast_face(path)
        # append all faces in an image into new faces array
        for pixel in face:
            faces.append(pixel)
    return faces

In [25]:
# load a dataset that contains one subdir  for each class that in turn contains images
def load_dataset(dir_):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in os.listdir(dir_):
        # path
        path = os.path.join(dir_, subdir)
        # skip any files that might be in the dir
        if not os.path.isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels 
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print(f"loaded {len(faces)} examples for class {subdir}")
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [26]:
# load train dataset
trainX, trainy = load_dataset("data/train")
print(trainX.shape, trainy.shape)

loaded 26 examples for class Abdul Rehman
loaded 14 examples for class ahmed
loaded 14 examples for class Ahmed Naeem
loaded 14 examples for class ben_afflek
loaded 18 examples for class elton_john
loaded 13 examples for class Fouzan
loaded 25 examples for class Hassan Qureshi
loaded 22 examples for class jerry_seinfeld
loaded 17 examples for class madonna
loaded 22 examples for class mindy_kaling
loaded 14 examples for class Mohib Rehman
loaded 10 examples for class Shayan
loaded 7 examples for class Sumair
loaded 29 examples for class Zulfiqar
(245, 224, 224, 3) (245,)


In [27]:
testX, testy = load_dataset("data/val")

loaded 5 examples for class Abdul Rehman
loaded 11 examples for class ahmed
loaded 7 examples for class Ahmed Naeem
loaded 5 examples for class ben_afflek
loaded 4 examples for class elton_john
loaded 11 examples for class Fouzan
loaded 5 examples for class Hassan Qureshi
loaded 7 examples for class jerry_seinfeld
loaded 5 examples for class madonna
loaded 5 examples for class mindy_kaling
loaded 4 examples for class Mohib Rehman
loaded 2 examples for class Shayan
loaded 3 examples for class Sumair
loaded 7 examples for class Zulfiqar


In [28]:
savez_compressed('new_data.npz', trainX, trainy, testX, testy)

In [6]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(model,faces):
    # convert into an array of samples
    samples = faces.astype('float32')
    samples = expand_dims(samples, axis=0)
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # perform prediction
    yhat = model.predict(samples)
    return yhat[0]

In [67]:
# load the face dataset
data = load('new_data.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print(f"Loaded: {trainX.shape, trainy.shape, testX.shape, testy.shape}")
# load the facenet model
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embeddings(model,face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    break
    embedding = get_embeddings(model,face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

Loaded: ((245, 224, 224, 3), (245,), (81, 224, 224, 3), (81,))
(224, 224, 3)
(0,)
(0,)


In [35]:
savez_compressed('face_embeddings_new.npz', newTrainX, trainy, newTestX, testy)

In [8]:
data = load('face_embeddings_new.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print(f"Dataset: train={trainX.shape[0], testX.shape[0]}")
#normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
save('classes_new.npy', out_encoder.classes_)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print(f"Accuracy: train={score_train*100}, test={score_test*100}")

Dataset: train=(245, 81)
Accuracy: train=99.59183673469387, test=95.06172839506173


In [9]:
import pickle
filename = 'finalized_new_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [10]:
import pickle
filename = 'finalized_new_model.sav'
model1 = pickle.load(open(filename, 'rb'))
encoder = LabelEncoder()
encoder.classes_ = load('classes_new.npy')
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [30]:
def run(filename, required_size=(224,224)):
    img = cv2.imread(filename)
    img_rgb = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    boxes, _ = mtcn.detect(img_rgb)
    name = None
    conf = 0.95
    prob_= 35.00
    for c, i in enumerate(_):
        data = list()
        if i > 0.95:
            box = boxes[c]
            f = img_rgb.crop((box[0], box[1], box[2], box[3]))
            image = asarray(f.resize(required_size))
            data.append(get_embeddings(model, image))
            data = asarray(data)
            in_encoder = Normalizer(norm='l2')
            dataX = in_encoder.transform(data)
            class_ = model1.predict(dataX)
            yhat_prob = model1.predict_proba(dataX)
            predict_names = encoder.inverse_transform(class_)
            prob = yhat_prob[0, class_]*100
            if prob[0] > prob_:
                print(prob[0], predict_names[0])
                name = predict_names
                cv2.putText(img, f'{name[0]}', (int(box[0]), int(box[1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)
    cv2.imshow('faces', img)
    cv2.waitKey(0) 

In [31]:
run("testing_data/ahmed1.1.jpeg")

In [26]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
 
# define filenames
filenames = ['testing_data/ahmed_new.jpg', 'testing_data/ahmed1.1.jpeg', 'testing_data/fouz.jpeg', 'testing_data/fouzan.jpeg','testing_data/ahmed1.2.jpeg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
print(embeddings[0].shape)
print(embeddings[1].shape)
# # define sharon stone
# sharon_id = embeddings[0]
# # verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[4])
# is_match(embeddings[0], embeddings[2])em
# # verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[2])

[{'box': [587, 280, 221, 286], 'confidence': 1.0, 'keypoints': {'left_eye': (666, 384), 'right_eye': (767, 387), 'nose': (726, 441), 'mouth_left': (674, 497), 'mouth_right': (760, 498)}}]
[{'box': [504, 697, 169, 298], 'confidence': 0.9475782513618469, 'keypoints': {'left_eye': (621, 822), 'right_eye': (671, 817), 'nose': (670, 888), 'mouth_left': (609, 943), 'mouth_right': (645, 940)}}]
[{'box': [47, 32, 43, 48], 'confidence': 0.9989558458328247, 'keypoints': {'left_eye': (61, 53), 'right_eye': (81, 52), 'nose': (72, 63), 'mouth_left': (63, 71), 'mouth_right': (81, 71)}}]
[{'box': [357, 71, 112, 137], 'confidence': 1.0, 'keypoints': {'left_eye': (389, 123), 'right_eye': (443, 123), 'nose': (415, 147), 'mouth_left': (390, 173), 'mouth_right': (438, 174)}}]
[{'box': [31, 67, 36, 39], 'confidence': 0.9999871253967285, 'keypoints': {'left_eye': (43, 82), 'right_eye': (60, 79), 'nose': (54, 91), 'mouth_left': (46, 98), 'mouth_right': (62, 96)}}]
(2048,)
(2048,)
Positive Tests
>face is NOT 

In [30]:
trainy = [0,0,1,1,0]
model = SVC(kernel='linear', probability=True)
model.fit(embeddings, trainy)
# predict
yhat_train = model.predict(embeddings)
yhat_test = model.predict(embeddings)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(trainy, yhat_test)

# summarize
print(f"Accuracy: train={score_train*100}, test={score_test*100}")

Accuracy: train=100.0, test=100.0


In [32]:
yhat_test

array([0, 0, 1, 1, 0])